In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

In [2]:
# Load a pre-trained Sentence Transformer model from Hugging Face
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [3]:
# Function to compute sentence embeddings
def compute_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    # Use mean pooling for embeddings
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.detach().numpy()

# Sample documents (corpus) for semantic search
documents = [
    "The quick brown fox jumps over the lazy dog.",
    "Artificial intelligence is transforming the world.",
    "Python is a popular programming language for data science.",
    "Machine learning models can analyze large datasets.",
    "Transformers are state-of-the-art models in NLP."
]

# Compute embeddings for documents
doc_embeddings = np.vstack([compute_embedding(doc) for doc in documents])

# Semantic search function
def semantic_search(query, documents, doc_embeddings, top_k=3):
    query_embedding = compute_embedding(query)
    similarities = np.dot(doc_embeddings, query_embedding.T).flatten()
    top_indices = similarities.argsort()[::-1][:top_k]
    return [(documents[i], similarities[i]) for i in top_indices]


In [4]:
# Example query
query = "Tell me about deep learning and transformers."

# Perform semantic search
results = semantic_search(query, documents, doc_embeddings)

# Display results
print("Query:", query)
print("\nTop Results:")
for rank, (doc, score) in enumerate(results, start=1):
    print(f"{rank}. {doc} (Score: {score:.4f})")

Query: Tell me about deep learning and transformers.

Top Results:
1. Artificial intelligence is transforming the world. (Score: 20.7847)
2. Transformers are state-of-the-art models in NLP. (Score: 19.2917)
3. Machine learning models can analyze large datasets. (Score: 11.2499)
